<a href="https://colab.research.google.com/github/paraery/parsingResume/blob/main/spacyBertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textfortraining8.csv')
lstTxt = df['text'].values.tolist()

In [22]:
#train=[]
for a in lstTxt:
    train.append(get_tokens_with_entities(a))

In [23]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/testData.csv')
lstTest = df['text'].values.tolist()

In [3]:
import re
def get_tokens_with_entities(raw_text: str):
    # split the text by spaces only if the space does not occur between square brackets
    # we do not want to split "multi-word" entity value yet
    
    raw_tokens = re.split(r"\s(?![^\[]*\])", raw_text)

    # a regex for matching the annotation according to our notation [entity_value](entity_name)
    entity_value_pattern = r"\[(?P<value>.+?)\]\((?P<entity>.+?)\)"
    entity_value_pattern_compiled = re.compile(entity_value_pattern, flags=re.I|re.M)

    tokens_with_entities = []
    lstEnt=[]
    txt = raw_text
    for raw_token in raw_tokens:
        match = entity_value_pattern_compiled.match(raw_token)
        if match:
            raw_entity_name, raw_entity_value = match.group("entity"), match.group("value")
            
            # we prefix the name of entity differently
            # B- indicates beginning of an entity
            # I- indicates the token is not a new entity itself but rather a part of existing one
            #txt = raw_text[raw_text.find(str(raw_entity_value))-1 : ]
            txt = raw_text.replace(raw_text[int(raw_text.find(raw_entity_value))-1],'',1)
            txt = txt.replace('](PERSON)','')
            #print(txt)
            lstdata = ()
            entity_name = f"{raw_entity_name}"
            entName = createSubEntity(txt.find(raw_entity_value), txt.find(raw_entity_value) + len(raw_entity_value), entity_name)
            lstEnt.append(entName)
                #tokens_with_entities.append((raw_entity_token, entity_name))
    d={"entities": lstEnt}
            #print(lstEnt)
    lstdata=(txt,d)
    tokens_with_entities.append(lstdata)
    return lstdata
def createSubEntity(start,end,tag):
    s = (int(start),int(end),tag)
    return s

In [25]:
len(train),len(test)

(620, 87)

In [24]:
test=[]
for a in lstTest:
    test.append(get_tokens_with_entities(a))

In [23]:
import json
jsonStr = json.dumps(train_data)
import json
with open('drive/MyDrive/Spacy/train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

In [26]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.7 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.


In [27]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [28]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [30]:
!python -m spacy init fill-config /content/drive/MyDrive/Spacy1/base_config.cfg /content/drive/MyDrive/Spacy1/config.cfg

2023-03-04 14:56:26.766691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 14:56:26.766794: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 14:56:26.766812: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Spacy1/config.cfg
You can now add your data and train your pipeline:
python -m spacy train c

In [29]:
def get_spacy_doc(file,data):
  nlp = spacy.blank("en")
  db = DocBin()
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']
    ents = []
    entity_indices = []
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity==True:
        continue
      entity_indices = entity_indices+list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
      except:
        continue
      if span is None:
        err_data = str([start,end])+" "+str(text)+"\n"
        file.write(err_data)
      else:
        ents.append(span)
      
      doc.ents = ents
      db.add(doc)
  return db

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [31]:
file = open('drive/MyDrive/Spacy/error.txt','w')
db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 87/87 [00:00<00:00, 159.08it/s]


In [32]:
!python -m spacy train /content/drive/MyDrive/Spacy1/config.cfg --output ./outputSpacybert --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-03-04 14:57:05.446340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 14:57:05.446434: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 14:57:05.446453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Created output directory: outputSpacybert
ℹ Saving to output directory: outputSpacybert
ℹ Using GPU: 0

=========================== Initializing pipeline ===========

In [44]:
!zip -r /content/file.zip /content/outputSpacybert

  adding: content/outputSpacybert/ (stored 0%)
  adding: content/outputSpacybert/model-last/ (stored 0%)
  adding: content/outputSpacybert/model-last/tokenizer (deflated 81%)
  adding: content/outputSpacybert/model-last/meta.json (deflated 57%)
  adding: content/outputSpacybert/model-last/ner/ (stored 0%)
  adding: content/outputSpacybert/model-last/ner/moves (deflated 43%)
  adding: content/outputSpacybert/model-last/ner/cfg (deflated 33%)
  adding: content/outputSpacybert/model-last/ner/model (deflated 8%)
  adding: content/outputSpacybert/model-last/config.cfg (deflated 61%)
  adding: content/outputSpacybert/model-last/transformer/ (stored 0%)
  adding: content/outputSpacybert/model-last/transformer/cfg (stored 0%)
  adding: content/outputSpacybert/model-last/transformer/model (deflated 8%)
  adding: content/outputSpacybert/model-last/vocab/ (stored 0%)
  adding: content/outputSpacybert/model-last/vocab/vectors (deflated 45%)
  adding: content/outputSpacybert/model-last/vocab/lookup

In [45]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#TEST


In [33]:
nlp = spacy.load('outputSpacybert/model-best')

In [34]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textInResume.csv')
dataTest = df['text'].values.tolist()

In [35]:
import re
i=0

bst_pred=[]
for d in dataTest:
  doc = nlp(d)
  print("## doc ",i," ##")
  person = ''
  for ent in doc.ents:
    nameRegex = re.compile('[A-Za-z]{2,25} [A-Za-z]{2,25}')
    if nameRegex.findall(ent.text)!=[]:
      print(ent.text)
      person=ent.text
  bst_pred.append(person)

    #print(ent.text,"----",ent.label_)
  i = i+1
  print("   ")

## doc  0  ##
supavit attagomol
   
## doc  1  ##
nisaratwisesbantao bachelor
   
## doc  2  ##
thirawit jirarungroj
   
## doc  3  ##
   
## doc  4  ##
thanchanok watcharakitphokin
   
## doc  5  ##
|natanop pimonsathian
   
## doc  6  ##
patcharaya anuntasinkul
   
## doc  7  ##
jarrukorn pensalaphan
   
## doc  8  ##
pravittra vimonworachort
   


Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors


## doc  9  ##
   
## doc  10  ##
purawat ruangsri
   
## doc  11  ##
nutratanon mahakhet
   
## doc  12  ##
kitisak thossaensin
   
## doc  13  ##
suebtas limsaihua
   
## doc  14  ##
   
## doc  15  ##
chayanit sripradit
   
## doc  16  ##
pimnares puto
   
## doc  17  ##
saran thitawiriyayos
   
## doc  18  ##
pakpoom poodsud
   
## doc  19  ##
kanittha setthapitayakul
   
## doc  20  ##
parweenuch duangdenphatsawron
   
## doc  21  ##
pawith panyasirikul
   
## doc  22  ##
chotitouch supalanunt
   
## doc  23  ##
aussadach masun
   
## doc  24  ##
teeratach jearapaganon
   
## doc  25  ##
pratan srikamonpattanawut
   
## doc  26  ##
   
## doc  27  ##
chayanon juntarapartsavorn
   
## doc  28  ##
narurong saeheng
   
## doc  29  ##
supachai sumeteenarumit
   
## doc  30  ##
ittichote sornmeethong
   
## doc  31  ##
kamolchai suebnipon
   
## doc  32  ##
   
## doc  33  ##
nattachai noogure
   
## doc  34  ##
saksorn pawasakarin
   
## doc  35  ##
navapuvadol uraikul
   
## doc  36  

In [36]:
bst_pred

['supavit attagomol',
 'nisaratwisesbantao bachelor',
 'thirawit jirarungroj',
 '',
 'thanchanok watcharakitphokin',
 '|natanop pimonsathian',
 'patcharaya anuntasinkul',
 'jarrukorn pensalaphan',
 'pravittra vimonworachort',
 '',
 'purawat ruangsri',
 'nutratanon mahakhet',
 'kitisak thossaensin',
 'suebtas limsaihua',
 '',
 'chayanit sripradit',
 'pimnares puto',
 'saran thitawiriyayos',
 'pakpoom poodsud',
 'kanittha setthapitayakul',
 'parweenuch duangdenphatsawron',
 'pawith panyasirikul',
 'chotitouch supalanunt',
 'aussadach masun',
 'teeratach jearapaganon',
 'pratan srikamonpattanawut',
 '',
 'chayanon juntarapartsavorn',
 'narurong saeheng',
 'supachai sumeteenarumit',
 'ittichote sornmeethong',
 'kamolchai suebnipon',
 '',
 'nattachai noogure',
 'saksorn pawasakarin',
 'navapuvadol uraikul',
 'harit piyapornthana',
 'setthawuth kangwansakol',
 'miss.ratchanok angkhahad',
 'veerachaimitmorn creativity',
 'phanchita korsanankittipat',
 'suphakit sangthong',
 '',
 'jitawat chan

In [37]:
nlpLst = spacy.load('outputSpacybert/model-last')

In [38]:
import re
i=0
lst_pred=[]
for d in dataTest:
  doc = nlpLst(d)
  print("## doc ",i," ##")
  person = ''
  for ent in doc.ents:
    nameRegex = re.compile('[A-Za-z]{2,25} [A-Za-z]{2,25}')
    if nameRegex.findall(ent.text)!=[]:
      print(ent.text)
      person=ent.text
  lst_pred.append(person)

    #print(ent.text,"----",ent.label_)
  i = i+1
  print("   ")

## doc  0  ##
supavit attagomol
   
## doc  1  ##
nisaratwisesbantao bachelor
   
## doc  2  ##
thirawit jirarungroj
   
## doc  3  ##
   
## doc  4  ##
thanchanok watcharakitphokin
   
## doc  5  ##
|natanop pimonsathian
   
## doc  6  ##
patcharaya anuntasinkul
   
## doc  7  ##
jarrukorn pensalaphan
   
## doc  8  ##
pravittra vimonworachort
   


Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors


## doc  9  ##
   
## doc  10  ##
purawat ruangsri
   
## doc  11  ##
nutratanon mahakhet
   
## doc  12  ##
kitisak thossaensin
   
## doc  13  ##
suebtas limsaihua
   
## doc  14  ##
   
## doc  15  ##
chayanit sripradit
   
## doc  16  ##
pimnares puto
   
## doc  17  ##
saran thitawiriyayos
   
## doc  18  ##
pakpoom poodsud
   
## doc  19  ##
kanittha setthapitayakul
   
## doc  20  ##
parweenuch duangdenphatsawron
   
## doc  21  ##
pawith panyasirikul
   
## doc  22  ##
chotitouch supalanunt
   
## doc  23  ##
aussadach masun
   
## doc  24  ##
teeratach jearapaganon
   
## doc  25  ##
pratan srikamonpattanawut
   
## doc  26  ##
   
## doc  27  ##
chayanon juntarapartsavorn
   
## doc  28  ##
narurong saeheng
   
## doc  29  ##
supachai sumeteenarumit
   
## doc  30  ##
ittichote sornmeethong
   
## doc  31  ##
kamolchai suebnipon
   
## doc  32  ##
   
## doc  33  ##
nattachai noogure
   
## doc  34  ##
saksorn pawasakarin
   
## doc  35  ##
navapuvadol uraikul
   
## doc  36  

In [39]:
lst_pred

['supavit attagomol',
 'nisaratwisesbantao bachelor',
 'thirawit jirarungroj',
 '',
 'thanchanok watcharakitphokin',
 '|natanop pimonsathian',
 'patcharaya anuntasinkul',
 'jarrukorn pensalaphan',
 'pravittra vimonworachort',
 '',
 'purawat ruangsri',
 'nutratanon mahakhet',
 'kitisak thossaensin',
 'suebtas limsaihua',
 '',
 'chayanit sripradit',
 'pimnares puto',
 'saran thitawiriyayos',
 'pakpoom poodsud',
 'kanittha setthapitayakul',
 'parweenuch duangdenphatsawron',
 'pawith panyasirikul',
 'chotitouch supalanunt',
 'aussadach masun',
 'teeratach jearapaganon',
 'pratan srikamonpattanawut',
 '',
 'chayanon juntarapartsavorn',
 'narurong saeheng',
 'supachai sumeteenarumit',
 'ittichote sornmeethong',
 'kamolchai suebnipon',
 '',
 'nattachai noogure',
 'saksorn pawasakarin',
 'navapuvadol uraikul',
 'harit piyapornthana',
 'setthawuth kangwansakol',
 '',
 'veerachaimitmorn creativity',
 'phanchita korsanankittipat',
 'suphakit sangthong',
 '',
 'jitawat chanpraneet',
 'anwar rajawa

In [40]:
import pandas as pd
dfName = pd.read_csv('drive/MyDrive/resumeData.csv')
lstTrue = dfName['PERSON'].values.tolist()

#Metrix of model best

In [41]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [42]:
y_true = np.array(lstTrue)
y_pred = np.array(bst_pred)

In [43]:
precision_recall_fscore_support(y_true, y_pred, average='weighted')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.7068965517241379, 0.7126436781609196, 0.7088122605363985, None)

In [44]:
precision_recall_fscore_support(y_true, y_pred, average='macro')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.6212121212121212, 0.6262626262626263, 0.6228956228956228, None)

In [45]:
precision_recall_fscore_support(y_true, y_pred, average='micro')

(0.7126436781609196, 0.7126436781609196, 0.7126436781609196, None)

In [46]:
precision_recall_fscore_support(y_true, y_pred, average=None)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. , 0. , 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. , 0. , 1. ,
        1. , 1. , 1. , 0. , 1. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. , 0. , 1. , 1. , 1. , 1. ,
        0. , 0. , 0. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 0. ,
        1. , 1. , 1. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 1. , 1. , 1. ,
        1. , 1. , 0. , 0. , 1. , 0. , 1. , 0. ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
        1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.,

#Metrix of model last

In [47]:
y_true = np.array(lstTrue)
y_pred = np.array(lst_pred)

In [48]:
precision_recall_fscore_support(y_true, y_pred, average='weighted')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.6724137931034483, 0.6781609195402298, 0.6743295019157088, None)

In [49]:
precision_recall_fscore_support(y_true, y_pred, average='macro')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.5909090909090909, 0.5959595959595959, 0.5925925925925927, None)

In [50]:
precision_recall_fscore_support(y_true, y_pred, average='micro')

(0.6781609195402298, 0.6781609195402298, 0.6781609195402298, None)

In [51]:
precision_recall_fscore_support(y_true, y_pred, average=None)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ,
        1. , 0. , 1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. , 0. , 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. , 0. , 1. , 1. ,
        1. , 1. , 0. , 1. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
        1. , 1. , 1. , 1. , 0. , 0. , 1. , 1. , 0. , 1. , 1. , 1. , 1. ,
        0. , 0. , 0. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 0. ,
        1. , 1. , 1. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 1. , 1. , 0. ,
        1. , 0. , 0. , 0. , 1. , 0. , 1. , 0. ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
        1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1.,